In [20]:
data=pd.read_csv('raw_data.csv')
data=data[['롯데첨단소재', '2019-09-11', '석식', '배식구3(INTER)']]
data.columns=['회사명', '식수일자',  '조식/중식/석식/야식', '식단구분']

,롯데첨단소재,의왕,2019-09-11,17:32:52,35124818,김영신,ABS)AD팀(OA),석식,배식구3(INTER),5000
0,롯데첨단소재,의왕,2019-09-11,17:33:24,35129357,김익모,PC)EP컴파운드팀,석식,배식구4(INTER),5000.0
1,롯데첨단소재,의왕,2019-09-11,17:46:57,35122221,김준명,PC)AD1팀(TV),석식,배식구3(INTER),5000.0
2,롯데첨단소재,의왕,2019-09-11,17:39:13,35132260,김혁진,마케팅지원팀,석식,배식구3(INTER),5000.0
3,롯데첨단소재,의왕,2019-09-11,17:41:57,35123852,김현철,칼라LAB,석식,배식구3(INTER),5000.0
4,롯데첨단소재,의왕,2019-09-11,17:45:15,35123840,민우,칼라LAB,석식,배식구6(라면),4287.0
...,...,...,...,...,...,...,...,...,...,...
292474,삼성SDI,의왕,2020-05-08,12:16:18,98161186,최민우,NaN,중식,배식구2(KOREAN),5000.0
292475,삼성SDI,의왕,2020-05-08,12:10:27,최신영,최신영,NaN,중식,배식구2(KOREAN),5000.0
292476,삼성SDI,의왕,2020-05-08,12:07:48,98160941,홍웅선,NaN,중식,배식구3(INTER),5000.0
292477,삼성SDI,의왕,2020-05-08,11:33:33,98180127,최준하,NaN,중식,배식구1(KOREAN),5000.0


In [9]:
a=data.groupby(['회사명','식수일자',  '조식/중식/석식/야식','식단구분']).count().reset_index()
data['식단구분']=data['식단구분'].apply(lambda x : str(x).replace('김밥','라면'))
data=data.rename(columns={'cnt':'식사인원'})
data.sort_values('식수일자').reset_index().drop('index',axis=1).to_csv('의왕점.csv',index=False)

In [52]:
import pandas as pd
data=pd.read_csv('의왕점.csv',index_col=False)
data=data[(data[ '조식/중식/석식/야식']=='중식')&(data['식단구분']=='KOREAN')].groupby('식수일자').sum().reset_index()
data=data.rename(columns={'식사인원':'target'})
data=data.rename(columns={'식수일자':'timestamp'})
data['timestamp']=data['timestamp'].apply(lambda x:pd.to_datetime(x))
data['dayofweek_mean']='.'
data['dayofweek_median']='.'

In [156]:
import calendar
data1['weekday']=data1['timestamp'].apply(lambda x: calendar.weekday(x.year,x.month,x.day))

In [157]:
data.groupby(['weekday']).median()
def func(total):
    if total['weekday']==0:
        return 217
    elif total['weekday']==1: 
        return 187
    elif total['weekday']==2: 
        return 172
    elif total['weekday']==3: 
        return 189
    elif total['weekday']==4: 
        return 151
    else:
        return 0
        pass 
    
data1.dayofweek_median=data1.apply(func,axis=1)

In [158]:
data.groupby(['weekday']).mean()
def func(total):
    if total['weekday']==0:
        return 205
    elif total['weekday']==1: 
        return 189
    elif total['weekday']==2: 
        return 176
    elif total['weekday']==3: 
        return 179
    elif total['weekday']==4: 
        return 154
    else:
        return 0
        pass 
    
data1.dayofweek_mean=data1.apply(func,axis=1)

In [56]:
# 과거 temperature    
from bs4 import BeautifulSoup
import requests
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=108&yy={year}&obs=08&x=32&y=9'.format(year='2018'))  
soup=BeautifulSoup(bam.text,'html.parser')
content=soup.find_all('td')
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=108&yy={year}&obs=08&x=32&y=9'.format(year='2019'))  
soup=BeautifulSoup(bam.text,'html.parser')
content1=soup.find_all('td')
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=108&yy={year}&obs=08&x=32&y=9'.format(year='2020'))  
soup=BeautifulSoup(bam.text,'html.parser')
content2=soup.find_all('td')
data['temperature']=data['timestamp'].apply(lambda x: content[3+(x.day-1)*13+x.month].text if x.year==2018 else (content1[3+(x.day-1)*13+x.month].text if x.year==2019 else content2[3+(x.day-1)*13+x.month].text ))

In [57]:
import numpy as np
def abnormal(prev):
    try: 
        if (prev['timestamp'].month in [6,7,8]) & (float(prev['temperature'])>=28):
            return 1
        elif (prev['timestamp'].month in [9,10,11]) & (float(prev['temperature'])>=5):
            return 1
        elif (prev['timestamp'].month in [12,1,2]) & (float(prev['temperature'])<=-5):
            return 1
        elif (prev['timestamp'].month in [3,4,5]) & (float(prev['temperature'])<=5):
            return 1
        else:
            return 0
    except:
        prev['temperature']==np.nan
            
data['abnormal']=data.apply(abnormal,axis=1) 

In [58]:
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=108&yy=2018&obs=21&x=18&y=7')  
soup=BeautifulSoup(bam.text,'html.parser')
content=soup.find_all('td')
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=108&yy=2019&obs=21&x=18&y=7')  
soup=BeautifulSoup(bam.text,'html.parser')
content1=soup.find_all('td')
bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=108&yy=2020&obs=21&x=18&y=7')  
soup=BeautifulSoup(bam.text,'html.parser')
content2=soup.find_all('td')
data['rainfall']=data['timestamp'].apply(lambda x: content[3+(x.day-1)*13+x.month].text if x.year==2018 else (content1[3+(x.day-1)*13+x.month].text if x.year==2019 else content2[3+(x.day-1)*13+x.month].text ))

In [142]:
holiday=[]
date=['01','02','03','04','05','06','07','08','09','10','11','12']
for j in range(2018,2021):
    for i in range(len(date)):
        url='http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?solYear={year}&solMonth={date}&ServiceKey=J%2FYE3sN528TkttbFXsmjXDfVrpZyKNjduV1r%2FcUxAfJ7wHnrjO2F5zzkJx2JvBHsmuNstAVe%2B5n6fS2dNgcKvQ%3D%3D'.format(year=str(j),date=date[i])
        resp=requests.get(url)
        soup=BeautifulSoup(resp.content)
        Text=soup.find_all('locdate')
        for i in range(len(Text)):
            holiday.append(pd.to_datetime(Text[i].text)) 
            
def holiday1(x):
    if (x in holiday)==False:
        return 0
    else:
        return 1
    
from datetime import timedelta    
def before_holiday(data):
    if data['holiday']==1:
        return 0
    elif data['timestamp']+timedelta(days=1) in holiday:
        return 1
    else:
        return 0    

def after_holiday(data):
    if data['holiday']==1:
        return 0
    elif data['timestamp']+timedelta(days=-1) in holiday:
        return 1
    else:
        return 0 
        
data1['holiday']=data1['timestamp'].apply(lambda x: holiday1(x))
data1['before_holiday']=data1.apply(before_holiday,axis=1)
data1['after_holiday']=data1.apply(after_holiday,axis=1)   

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [81]:
def z_score_normalize(lst):
    normalized = []
    for value in lst:
        normalized_num = (value - np.mean(lst)) / np.std(lst)
        normalized.append(normalized_num)
    return normalized


#prev=06/15 까지 채운 데이터프레임 
##오늘(6월 16일)부터 10일 간의 데이터 채워지는 함수입니다   -----> 까지 채우는 걸로 바꿔야ㅇㅇ
def make_df(prev,n):
    prev['timestamp']=pd.to_datetime(prev['timestamp'])
    from bs4 import BeautifulSoup
    import requests
    # 과거 rainfall
    bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=108&yy=2020&obs=21&x=18&y=7')  
    soup=BeautifulSoup(bam.text,'html.parser')
    content=soup.find_all('td')
    month=[i.month for i in prev[prev['rainfall'].notnull()]['timestamp']]
    day=[i.day for i in prev[prev['rainfall'].notnull()]['timestamp']]
    prev.loc[prev['rainfall'].notnull(),'rainfall']=[i.replace("\xa0", "0") if i =='\xa0' else i for i in [content[3+(day[u]-1)*13+month[u]].text for u in range(len(month))]]   
    
# 과거 temperature    
    bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=108&yy=2020&obs=08&x=25&y=4')  
    soup=BeautifulSoup(bam.text,'html.parser')
    content=soup.find_all('td')
    month=[i.month for i in prev[prev['temperature'].notnull()]['timestamp']]
    day=[i.day for i in prev[prev['temperature'].notnull()]['timestamp']]
    prev.loc[prev['temperature'].notnull(),'temperature']=[float(i) for i in [content[3+(day[u]-1)*13+month[u]].text for u in range(len(month))]]
    
# 미래    
#     bam=requests.get('https://m.weather.naver.com/')  
#     soup=BeautifulSoup(bam.text,'html.parser')
#     content=soup.find_all('div',{'class':'weather'})
#     prob=[]
#     for i in range(n):
#         prob.append(content[i].find_all('span',{'class':'percent'})[0].text[:-1]) 

#     bam=requests.get('https://m.weather.naver.com/')
#     soup=BeautifulSoup(bam.text,'html.parser')
#     content=soup.find_all('div',{'class':'weekly_item_temperature'})
#     temp=[]
#     for i in range(n):
#         temp.append(content[i].find_all('span')[3].text[6:8]) # 06/17~06/27

    import numpy as np
    from datetime import timedelta
    num=[]
    for i in range(1,n):
        num.append(int(prev.index[-1]+i))

    for i in range(n-1):    
        prev.loc[num[i]]=np.nan
        prev.loc[num[i],'timestamp']=prev.loc[num[i]-1,'timestamp']+timedelta(days=1)
        prev.loc[num[i],'day_cos']=prev.loc[num[i]-7,'day_cos']
        prev.loc[num[i],'day_sin']=prev.loc[num[i]-7,'day_sin']
        prev.loc[num[i],'weekday']=prev.loc[num[i]-7,'weekday']
        prev.loc[num[i],'dayofweek_median']=prev.loc[num[i]-7,'dayofweek_median']
        prev.loc[num[i],'dayofweek_mean']=prev.loc[num[i]-7,'dayofweek_mean']
          
#     from datetime import datetime
#     for i in range(n):
#         prev.loc[prev['timestamp']==datetime.strptime(datetime.today().strftime("%Y-%m-%d"),"%Y-%m-%d")+timedelta(days=i),'temperature']=temp[i] 
#         prev.loc[prev['timestamp']==datetime.strptime(datetime.today().strftime("%Y-%m-%d"),"%Y-%m-%d")+timedelta(days=i),'rain_prob']=int(prob[i])*0.1
    
    url='http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?solYear={year}&solMonth={date}&ServiceKey=J%2FYE3sN528TkttbFXsmjXDfVrpZyKNjduV1r%2FcUxAfJ7wHnrjO2F5zzkJx2JvBHsmuNstAVe%2B5n6fS2dNgcKvQ%3D%3D'.format(year=str((prev.loc[prev.index[-1],'timestamp']).year),date=str(prev.loc[prev.index[-1],'timestamp'].month))
    resp=requests.get(url)
    soup=BeautifulSoup(resp.content)
    Text=soup.find_all('locdate')
    holiday=[]
    for i in range(len(Text)):
        holiday.append(pd.to_datetime(Text[i].text))  
    def holiday1(prev):
        if prev['timestamp'] in holiday:
            return 1
        else:
            return 0
    def before_holiday(prev):
        if prev['holiday']==1:
            return 0
        elif prev['timestamp']+timedelta(days=1) in holiday:
            return 1
        else:
            return 0    

    def after_holiday(prev):
        if prev['holiday']==1:
            return 0
        elif prev['timestamp']+timedelta(days=-1) in holiday:
            return 1
        else:
            return 0 
    def abnormal(prev):
        try: 
            if (prev['timestamp'].month in [6,7,8]) & (int(prev['temperature'])>=28):
                return 1
            elif (prev['timestamp'].month in [9,10,11]) & (int(prev['temperature'])>=5):
                return 1
            elif (prev['timestamp'].month in [12,1,2]) & (int(prev['temperature'])<=-5):
                return 1
            elif (prev['timestamp'].month in [3,4,5]) & (int(prev['temperature'])<=5):
                return 1
            else:
                return 0
        except:
            prev['temperature']==np.nan
            
    prev['abnormal']=prev.apply(abnormal,axis=1)         
    prev['holiday']=prev.apply(holiday1,axis=1)
    prev['before_holiday']=prev.apply(before_holiday,axis=1)
    prev['after_holiday']=prev.apply(after_holiday,axis=1)   
    
    bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=108&yy=2020&obs=21&x=18&y=7')  # 과거 rainfall rain_prob
    soup=BeautifulSoup(bam.text,'html.parser')
    content=soup.find_all('td')
    month=[i.month for i in prev[prev['rainfall'].notnull()]['timestamp']]
    day=[i.day for i in prev[prev['rainfall'].notnull()]['timestamp']]
    prev.loc[prev['rainfall'].notnull(),'rainfall']=[i.replace("\xa0", "0") if i =='\xa0' else i for i in [content[3+(day[u]-1)*13+month[u]].text for u in range(len(month))]]   
#     prev.loc[(prev['rain_prob'].notnull())&(prev['rainfall']=='0'),'rain_prob']=0
    
    bam=requests.get('https://www.weather.go.kr/weather/climate/past_table.jsp?stn=108&yy=2020&obs=08&x=25&y=4')  # 과거 temperature
    soup=BeautifulSoup(bam.text,'html.parser')
    content=soup.find_all('td')
    month=[i.month for i in prev[prev['temperature'].notnull()]['timestamp']]
    day=[i.day for i in prev[prev['temperature'].notnull()]['timestamp']]
    prev.loc[prev['temperature'].notnull(),'temperature']=[i for i in [content[3+(day[u]-1)*13+month[u]].text for u in range(len(month))]]
   
#     if float(prev.loc[prev['abnormal'].isnull(),'temperature'])>=28:
#         prev.loc[prev['abnormal'].isnull(),'abnormal']=1
#     else:
#         prev.loc[prev['abnormal'].isnull(),'abnormal']=0
    
    return prev    


make_df(data,10).tail(20)

ValueError: could not convert string to float: 

In [113]:
import pandas as pd
start_date='2018-12-11'
end_date='2020-10-10'
daterange = pd.date_range(start_date, end_date)
l=[]
for single_date in daterange:
    l.append(single_date.strftime("%Y-%m-%d"))
l=pd.Series(l)    

In [136]:
d=pd.DataFrame(l,columns={'timestamp'})
d['timestamp']=d['timestamp'].apply(lambda x: pd.to_datetime(x))

In [134]:
data['timestamp']=data['timestamp'].apply(lambda x: pd.to_datetime(x))

In [138]:
data1=pd.merge(data,d,on='timestamp',how='right')
data1=data1.fillna(0)

In [237]:
data1[(data1.target==0)&(data1.weekday.isin([1,2,3,4]))&(data1.holiday==0)]['holiday'].index.values

array([], dtype=int64)

In [239]:
def z_score_normalize(lst):
    normalized = []
    for value in lst:
        normalized_num = (value - np.mean(lst)) / np.std(lst)
        normalized.append(normalized_num)
    return normalized
z_score_normalize(data1['target'])

[0.6019517727800754,
 0.37558447072504236,
 0.7313045168115228,
 0.8822160515148781,
 -1.2952218063478205,
 -1.2952218063478205,
 1.119362748905865,
 0.5049372147564898,
 0.14921716867000934,
 0.5372754007643517,
 0.11687898266214747,
 -1.2952218063478205,
 -1.2952218063478205,
 -1.2952218063478205,
 -1.2952218063478205,
 0.9900100048744177,
 0.23545233135764096,
 -0.10948831939288554,
 -1.2952218063478205,
 -1.2952218063478205,
 -1.2952218063478205,
 -1.2952218063478205,
 0.7097457261396148,
 1.2487154929373125,
 0.2677905173655028,
 -1.2952218063478205,
 -1.2952218063478205,
 1.1301421442418191,
 0.8390984701710623,
 0.5803929821081675,
 0.688186935467707,
 1.0654657722260954,
 -1.2952218063478205,
 -1.2952218063478205,
 1.5074209810002073,
 1.2918330742811284,
 0.23545233135764096,
 0.8714366561789242,
 0.1815553546778712,
 -1.2952218063478205,
 -1.2952218063478205,
 0.8283190748351084,
 0.5372754007643517,
 -0.2603998540962409,
 0.2031141453497791,
 0.4294814474048121,
 -1.29522180

In [ ]:
class ldccLstm():
    
    ### 초기 설정
    def __init__(self, **kwargs):
        
        ### warning 메시지 무시
        import warnings
        warnings.filterwarnings(action='ignore')
        
        
        ### aws python SDK 객체 생성 및 Amazon Forecast, Forecastquery, S3 에 연동, IAM 설정
        self.fcst = ldcc_forecast_01_3.ldccForecast()
        # region_name = ['서울': 'ap-northeast-2', '싱가포르': 'ap-southeast-1']
        self.fcst.region_name = 'ap-northeast-2'
        self.fcst.connect_aws()
        self.fcst.iam_config()

        
        ### 인자 임포트 및 선언
        with open('./bilstm_0609.yaml') as f:
            self.argData = yaml.load(f, Loader=yaml.FullLoader)

            # common
            self.train_start_date = self.argData['common']['train_start_date']
            self.train_end_date = self.argData['common']['train_end_date']
            self.test_start_date = self.argData['common']['test_start_date']
            self.test_end_date = self.argData['common']['test_end_date']
            self.modelName = self.argData['common']['modelName']
            
            # data_import
            self.train_readS3BucketName = self.argData['data_import']['train_readS3BucketName']
            self.train_readS3Key = self.argData['data_import']['train_readS3Key']  
            self.rel_readS3BucketName = self.argData['data_import']['rel_readS3BucketName']
            self.rel_readS3Key = self.argData['data_import']['rel_readS3Key']
            
            # data_prep
            self.n_steps_in = self.argData['data_prep']['n_steps_in']
            self.n_steps_out = self.argData['data_prep']['n_steps_out']
            self.features = self.argData['data_prep']['features']

            # train_model
            self.cell = self.argData['train_model']['cell']
            self.epochs = self.argData['train_model']['epochs']
            self.batch_size = self.argData['train_model']['batch_size']
            self.verbose = self.argData['train_model']['verbose']
            
            # save_res
            self.upload_to_s3_bool = self.argData['save_res']['upload_to_s3_bool']
            self.save_dir = self.argData['save_res']['save_dir']
            
        
    def data_prep(self):
        
        ## hstack 생성
        # 피처별 어레이 데이터를 담을 딕셔너리 선언
        feat_dict = {}
        
        # 피처가 아닌 칼럼 제거
        column_list = self.merged_df.columns.drop(['timestamp', 'item_id'])
        
        # 각 피처 별로 어레이 타입으로 딕셔너리 구성 후, hstack으로 합치기 
        for column_name in column_list:
            feat_dict['%s' %column_name] = array(
                self.merged_df['%s' %column_name]).reshape(
                (len(self.merged_df['%s' %column_name]),1))
        
        self.raw_dataset = hstack(feat_dict.values())
        
        print('### hstack 생성')
        for i in (0, 1, 2, 3, -3, -2, -1):
            print('# hstack[%s]' %i)
            print(self.raw_dataset[i])
            print('\n')        
        print('\n\n\n')
    
    
        ## 시퀀스 쪼개기
        # X, y로 쪼개기 위한 리스트 선언
        X, y = [], []

        # 시퀀스 길이만큼 1 step으로 수행
        for i in range(len(self.raw_dataset)):
            # X의 미만 위치, y의 이상 위치
            end_ix = i + self.n_steps_in
            # y의 미만 위치
            out_end_ix = end_ix + self.n_steps_out
            # y의 미만 위치가 시퀀스 길이를 넘으면 종료
            if out_end_ix > len(self.raw_dataset):
                break
            # 시퀀스에서 multistep으로 X, y를 쪼개기
            x_seq, y_seq = self.raw_dataset[i:end_ix], self.raw_dataset[end_ix:out_end_ix, -1]
            X.append(x_seq)
            y.append(y_seq)

        self.X = array(X)
        self.y = array(y)
        
        self.train_X = self.X[4:]
        self.train_y = self.y[4:]
        self.val_X = self.X[:4]
        self.val_y = self.y[:4]
        
        print('### sequence 쪼개기')
        for i in (0, 1, -2, -1):
            print('# train X[%s]' %i)
            print(self.train_X[i])
            print('# train y[%s]' %i)
            print(self.train_y[i])            
            print('\n')        
            
        for i in (0, 1, -2, -1):
            print('# validation X[%s]' %i)
            print(self.val_X[i])
            print('# validation y[%s]' %i)
            print(self.val_y[i])            
            print('\n')              
        print('\n\n\n')
    
    
    
    ### 모델 학습
    def train_model(self):        
        # 모델 정의                
        self.model = Sequential()
        self.model.add(Bidirectional(LSTM(self.cell, activation='relu'), input_shape=(self.n_steps_in, self.n_features)))
        self.model.add(Dense(self.n_steps_out))
        self.model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
        self.model.summary()
        print('\n\n')

        # 모델 fit
        print('### model 학습')
        hist = self.model.fit(self.train_X, self.train_y, 
                              validation_data=(self.val_X, self.val_y), 
                              epochs=self.epochs, 
                              batch_size = self.batch_size,
                              verbose=self.verbose)
        
        fig, loss_ax = plt.subplots()

        acc_ax = loss_ax.twinx()

        loss_ax.plot(hist.history['loss'], 'y', label='train loss')
        loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

        acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
        acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')        
        
        loss_ax.set_xlabel('epoch')
        loss_ax.set_ylabel('loss')
        acc_ax.set_ylabel('accuracy')

        loss_ax.legend(loc='upper left')
        acc_ax.legend(loc='lower left')
        
        print('\n\n\n')
        
    def run_pred(self):
        self.test_start_date = datetime.datetime.strptime(self.test_start_date, '%Y-%m-%d')           
        self.test_end_date = datetime.datetime.strptime(self.test_end_date, '%Y-%m-%d')         
        ## input_df 선언
        input_start_date = self.test_start_date + datetime.timedelta(days=-(self.n_steps_in+3))
        print('input_start_date: ', input_start_date)
        input_end_date = self.test_start_date + datetime.timedelta(days=-1)
        self.input_df = self.merged_df[
            (self.merged_df['timestamp'] >= input_start_date) 
            &
            (self.merged_df['timestamp'] <= input_end_date)
        ]
        print('### input_df 선언')        
        print('# input_df shape')
        print(self.input_df.shape)
        print()
        print('# input_df head')
        print(self.input_df.head())
        print()
        print('# input_df tail')
        print(self.input_df.tail())
        print('\n\n\n')
        
        ## input_x_seq 선언
        # hstack 생성
        # 피처별 어레이 데이터를 담을 딕셔너리 선언
        feat_dict = {}
        # 피처가 아닌 칼럼 제거
        column_list = self.input_df.columns.drop(['timestamp', 'item_id'])
        # 각 피처 별로 어레이 타입으로 딕셔너리 구성 후, hstack으로 합치기 
        for column_name in column_list:
            feat_dict['%s' %column_name] = array(
                self.input_df['%s' %column_name]).reshape(
                (len(self.input_df['%s' %column_name]),1))
        self.input_raw_dataset = hstack(feat_dict.values())
        print('### hstack 생성')
        print('# hstack[0]')
        print(self.input_raw_dataset[0])
        print('\n\n\n')
        # 시퀀스로 생성        
#         self.input_x_seq = array(self.input_raw_dataset).astype(np.int64)
#         self.input_x_seq = self.input_x_seq.reshape(1,self.input_x_seq.shape[0],self.input_x_seq.shape[1])
        self.input_x_seq = array(self.input_raw_dataset)
        self.input_x_seq = self.input_x_seq.reshape(1,self.input_x_seq.shape[0],self.input_x_seq.shape[1])
        print('### input_x 선언')
        print('# input_x_seq shape')
        print(self.input_x_seq.shape)
        print('\n') 
        print('# input_x_seq')
        print(self.input_x_seq)
        print('\n\n\n')
        
        
        ## 예측 수행
        self.yhat = self.model.predict(self.input_x_seq, verbose=0).astype(np.int64)      
        print('### 예측 수행')
        print('# yhat')
        print(self.yhat)
        print('\n\n\n')
        
        
        ## rst_df 생성
        # rst_df 1일 간격 날짜 변수 선언
        rst_date_var = self.test_start_date
        # 날짜를 입력할 리스트
        rst_date_list = []
        
        while rst_date_var != self.test_end_date + datetime.timedelta(days=1):
            rst_date_list.append(rst_date_var)
            rst_date_var += datetime.timedelta(days=1)
        
        self.rst_df = pd.DataFrame(rst_date_list, columns=['timestamp'])
        
        ## rst_df에 yhat 칼럼 추가
        self.rst_df['yhat'] = self.yhat[0][3:]
        print('### rst_df 생성')
        print('# rst_df')
        print(self.rst_df)        
        print('\n\n\n')
        
        ## y가 있다면 rst_df에 label 칼럼 선언               
        if self.target_df.iloc[-1,:]['timestamp'] > self.test_end_date:
            self.label = self.target_df[
                (self.target_df['timestamp'] >= self.test_start_date) 
                &
                (self.target_df['timestamp'] <= self.test_end_date)
            ][['target_value']]
            
            self.rst_df['label'] = list(self.label.target_value)
            
            print('### rst_df에 label 칼럼 추가')        
            print('# label')
            print(self.label)
            print('\n')
            print('# rst_df')
            print(self.rst_df)
            print('\n\n\n')            
        else:
            print('### label_df 선언')        
            print('- 예측하고자 하는 기간의 실제값이 없으므로 label_df 선언 X')
            print('# rst_df')
            print(self.rst_df)            
            print('\n\n\n')


    ### 라인그래프 시각화
    def visualize_rst(self):
        figure, ((ax1)) = plt.subplots(nrows=1, ncols=1)
        figure.set_size_inches(18,8)

        #legend list 선언
        legend_list = []    

        col_list = list(self.rst_df.columns)
        col_list.remove('timestamp')
        for line in col_list:
            sns.lineplot(x='timestamp', 
                         y='%s' %line, 
                         data=self.rst_df, 
                         linewidth=4,
                         label='big',
                         ax=ax1
                        )

        # legend list 붙이기
        legend_list.append('lunch' + ' ' + line)

        # legend 생성
        plt.legend(title='Line', loc='upper right', 
                   fontsize='x-large', title_fontsize='50', 
                   labels=legend_list)                        

        # x, y limit 설정    
        xlim = [self.rst_df['timestamp'][0], self.rst_df['timestamp'][-1:]]
        ax1.set_xlim(xlim)    
        ylim = [0,600]
        ax1.set_ylim(ylim)

        # x, y 라벨 이름, 크기
        ax1.set_xlabel('Date',fontsize=40);
        ax1.set_ylabel('Count',fontsize=40);                

        # major tick 폰트 사이즈 설정
        ax1.tick_params(axis='both', which='major', labelsize=30)


        # figure를 변수로 선언
        self.figure = figure
        
        
        
    ### error metrics 계산        
    def error_metric(self):
        
        # y가 있다면
        if self.target_df.iloc[-1,:]['timestamp'] > self.test_end_date:

            ## RMSE
            label = self.rst_df['label']
            yhat = self.rst_df['yhat']
            self.rmse = mean_squared_error(label, yhat)**0.5
            print('### RMSE')
            print(self.rmse)
            print('\n\n')

            ## MAPE
            # 타우가 0.5일 때
            # 분자 리스트 선언
            numer_list = []
            for i in range(0, len(list(self.rst_df))):
                # 실제값 - 예측값 
                abs_sub_val = abs(list(yhat)[i] - list(label)[i])                
            
                numer_list.append(abs_sub_val)
            
            total_numerator = sum(numer_list)
            total_denominator = sum(list(label))
            
            self.mape = 2 * (total_numerator / total_denominator)
            
            print('### MAPE by AWS')
            print(self.mape)
            print('\n\n')
            
            ## error_metric_df
            error_list = []
            error_list.append(self.modelName)
            error_list.append(self.rmse)
            error_list.append(self.mape)
            error_list
            
            self.error_df = pd.DataFrame([error_list], 
                                    columns=['model_name',
                                            'rmse',
                                            'mape'])
            
            print('### error_df 생성')
            print(self.error_df)
            print('\n\n\n')

        else:
            print('예측하고자 하는 구간의 실제값이 없으므로 error_metric 생략')
            print('\n\n\n')
            pass
        
        return True
    
if __name__ == '__main__':

    ### 0. convlstm 객체 생성
    ldcclstm = ldccLstm()    
    
    ### 2. 데이터 전처리
    ldcclstm.data_prep()
    
    ### 3. 모델 학습
    ldcclstm.train_model()
    
    ### 4. 예측 수행
    ldcclstm.run_pred()

    ### 6. 에러 메트릭 생성
    ldcclstm.error_metric()
    

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
X=pd.read_csv('input2_daysin.csv')
y=X[['target_value']]
X=pd.concat([X[X.columns.difference(['weekday','timestamp','target_value'])], pd.get_dummies(X['weekday'])],axis=1)

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X=scaler.fit_transform(X)
y=scaler.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1, random_state=2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(601, 16)
(67, 16)
(601, 1)
(67, 1)


In [10]:
# 학습 데이터, 테스트 데이터로 나누기 전에 fit과 transform 으로 변환. fit_transform() 도 무방. 
# scale된 iris_data로 학습과 테스트 데이터를 나눔.

In [11]:
X_train=X_train.reshape((-1,1,16))
X_test=X_test.reshape((-1,1,16))
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
# print(X_train.shape)
# print(X_test.shape)
# print(y_train.shape)
# print(y_test.shape)
# (1395, 49, 1)
# (599, 49, 1)
# (1395, 46)
# (599, 46)

(601, 1, 16)
(67, 1, 16)
(601, 1)
(67, 1)


In [24]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Input, Dense
from tensorflow.keras.layers import Input, Dense, LSTM, Bidirectional, Activation
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

model = Sequential()
model.add(Bidirectional(LSTM(70, return_sequences = False), input_shape = (1,16)))
# model.add(Bidirectional(LSTM(20, return_sequences = False), input_shape = (1,16)))
model.add(Dense(1))
model.add(Activation('softmax'))
adam = optimizers.Adam(lr = 0.001)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy']) 

# model = KerasClassifier(build_fn = bidirectional_lstm, epochs = 200, batch_size = 50, verbose = 2)
model.fit(X_train, y_train,epochs = 200, batch_size = 50, verbose = 2)
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)
print(accuracy_score(y_pred, y_test_)) 
RMSE = mean_squared_error(y_test, y_pred)**0.5
RMSE

Epoch 1/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 2/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 3/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 4/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 5/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 6/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 7/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 8/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 9/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 10/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 11/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 12/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 13/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 14/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 15/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 16/200
13/13 - 0s - loss: 0.0000e+00 - accuracy: 0.0017
Epoch 17/200
13/1

0.7358378701097333

In [ ]:
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Input, Dense
from tensorflow.keras.layers import Input, Dense, LSTM, Bidirectional, Activation

# train_X=pd.concat([train[train.columns.difference(['weekday','timestamp','target_value'])] ,pd.get_dummies(train['weekday'])],axis=1)
# train_X=scaler.fit_transform(train_X)
# train_X=train_X.reshape(-1,1,16)
# train_y=scaler.fit_transform(np.array(train['target_value']).reshape(-1,1)).reshape((-1,1))

# scaler=MinMaxScaler()
# val_X=pd.concat([test[test.columns.difference(['weekday','timestamp','target_value'])] ,pd.get_dummies(test['weekday'])],axis=1)
# val_X=scaler.fit_transform(val_X)
# val_X=val_X.reshape(-1,1,17)
# val_y=scaler.fit_transform(np.array(test['target_value']).reshape(-1,1)).reshape((-1,1))

# n_steps_in=7
# n_steps_out=7
# cell=8
# n_features=len(train_X.columns)

model = Sequential()
model.add(Bidirectional(LSTM(10, return_sequences=True, activation='sigmoid'), input_shape=(16,1)))
model.add(Bidirectional(LSTM(10)))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

hist = model.fit(train_X, train_y, validation_data=(val_X, val_y), epochs=50, batch_size = 20)
hist